In [2]:
import scipy.stats as ss
import numpy as np
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

In [3]:
data = np.array([6.2, 4.8, 5.3, 5.5, 5.1, 4.9, 4.7, 7.9, 6.6, 7.3])

In [3]:
sample1_mean = 5.0
sample1_std = 120

sample2_mean = 4450
sampe2_std = 120

 ![検定早見表](images/スクリーンショット.png)

![メソッド一覧](images/how_to_use_method_ss.png)

以下は母平均に関する検定である。(z, t検定)

# z検定
母標準偏差既知の場合

## 95%信頼区間推定
alpha=信頼区間, loc=期待値（平均）, 
scale=統計量(標準偏差/√自由度（標本サイズ）)

In [8]:
ss.norm.interval(alpha=0.95, loc=5, scale=1/np.sqrt(len(data)))
#ss.norm.interval(alpha=0.95, loc=np.mean(data), scale=np.std(data)/np.sqrt(len(data)))
np.mean(data)

5.83

## cdf(x=確率変数, loc=期待値, scale=標準偏差)
累積分布関数

In [9]:
ss.norm.cdf(ss.zscore(data))

array([0.63609918, 0.16629764, 0.30904469, 0.37811922, 0.24613721,
       0.19083274, 0.14389833, 0.97424498, 0.76556628, 0.91663667])

## ppf(q=パーセント点, loc=期待値, scale=標準偏差)
パーセント点関数から正規分布の信頼区間の棄却点を導き出す

In [10]:
alpha = 0.05
z_critical_value = ss.norm.ppf(alpha/2)
print("z critical value: [{:.3f}, {:.3f}]".format(z_critical_value, -z_critical_value))

z critical value: [-1.960, 1.960]


## z値の算出

In [11]:
z = (sample1_mean - sample2_mean) / (sample1_std / np.sqrt(50))
print("Criticulate z value: {:.3f}".format(z))

Criticulate z value: 2.946


## p値の算出

In [11]:
p_critical = ss.norm.cdf(z)
print("P value: {:}".format(1-p_critical))

NameError: name 'z' is not defined

# F分布 等分散性検定
2つの母集団の分散の比の検定によく使われる。  
自由度(n1, n2)

In [77]:
X = [0.3, 9.5, 13.6, 12.5, 11.6, 9.87, 9.4, 12.6, 11.6, 10.5]
Y = [16.2, 11.6, 12.4, 14.1, 10.8, 13.8, 14.6, 15.4, 19.1, 13.9]
F = np.var(X) / np.var(Y)
df1 = len(X) - 1
df2 = len(Y) - 1
alpha = 0.05
p_value = ss.f.cdf(F, df1, df2)
print("p value: {:.3f}".format(p_value))

p value: 0.901


In [74]:
a1 = np.array([9.5,9.7,10.1,9.8,9.3])
a2 = np.array([10.1,10.5,9.6,9.3])
a3 = np.array([11.3,10.7,10.2])

res = ss.f_oneway(a1,a2,a3)
print(res)

F_onewayResult(statistic=5.400987834994967, pvalue=0.028767554259032706)


## 水準間変動、残差変動、全変動
![変動](images/図6.png)  

In [55]:
allline = np.asarray([a1,a2,a3]).reshape(-1)
all_mean = np.mean(allline)
a1_mean = np.mean(a1)
a2_mean = np.mean(a2)
a3_mean = np.mean(a3)

total_var = np.sum([(i - all_mean)**2 for i in allline])
level_var = np.sum((a1_mean - all_mean)**2 * len(a1) + (a2_mean - all_mean)**2 * len(a2) + (a3_mean - all_mean)**2 * len(a3))
residual_var = total_var - level_var

print("水準間変動: {:.2f}\n 残差変動: {:.2f}\n 全変動: {:.2f}".format(level_var, residual_var, total_var))

水準間変動: 62.07
 残差変動: 83.40
 全変動: 145.47


# t検定(t分布)
母標準偏差が未知の場合

## パーセント点算出  
ppf 累積分布関数の逆関数  
ppf(0.975, 5)  
ppf(パーセント点, 自由度(n-1))

In [12]:
#ss.t.ppf(0.975, 5)
#(155.25-147.4)/
X = [160,168,158,165,161]
Y = [153,155,149,152]

2 つの母集団の<b>分散</b>が<b>同じとき</b>の t 検定  
t_ss, prob = ss.ttest_ind(
[データ1], [データ2], equal_var=True)  
片側検定の場合、p値を1/2する

In [14]:
t_ss, prob = ss.ttest_ind(X,Y, equal_var=False)
print("t value by scipy: {}".format(t_ss))
print("p value by scipy: {}".format(prob))

t value by scipy: 4.6219998185251905
p value by scipy: 0.0027120104740654682


2 つの母集団の<b>分散</b>が<b>同じでないとき</b>の t 検定
t_ss, prob = ss.ttest_ind([データ１], [データ２], equal_var=True)  
片側検定の場合、p値を1/2する

In [14]:
t_ss, prob = ss.ttest_ind([1326,1418,1820,1516,1635,1720,1580,1452,1600], [1220,1080,980,1420,1170,1290,1116], equal_var=False)
print("t value by scipy: {}".format(t_ss))
print("p value by scipy: {}".format(prob))

t value by scipy: 5.080348617271814
p value by scipy: 0.00019131443517208103


## 従属な 2 群間 t 検定  
> 例:ラットに薬剤を投与し、投与前と投与後の体重を測定したデータでは、それぞれのラットに対して投与前と投与後のデータがあるので、投与前と投与後のデータが対応しているといえる。  

つまり、同じ長さの配列が必要

In [4]:
t_ss, prob = ss.ttest_rel([1326,1418,1820,1516,1635,1720,1580,1452,1600], [1220,1080,980,1420,1170,1290,1116])
print("t value by scipy: {}".format(t_ss))
print("p value by scipy: {}".format(prob))

ValueError: unequal length arrays

# 二項検定

## 二項検定
p = ss.binom_test(当たり回数, 試行回数, 確率)
p = ss.binom_test(16, 20, 0.5)

In [7]:
ss.binom_test(16, 20, 0.5)

0.011817932128906248

# X^2検定

## パーセント点算出
ss.chi2.ppf(パーセント点, 自由度(n-1))
![カイニ乗検定1](images/図3.png)
![カイニ乗検定2](images/図4.png)
ss.chi2.ppf(0.95, 5)

In [ ]:
ss.chi2.ppf()

二次元以上のchi2検定
> 次の表は、ある小学校の1年生と年生に好きなスポーツを質問した結果である。この結果から、1年生と2年生のスポーツの好みに関連があるかどうかを有意水準0.05で検定せよ。
![カイニ乗検定3](images/図5.png)

In [72]:
array = np.array([[9,3],[18,30]])
result = ss.chi2_contingency(array)
print("test statistic: ", result[0])
print("p-value:", result[1])

test statistic:  4.044612794612794
p-value: 0.044312543173412496


## x^2検定
理論値  
ss.stats.chisquare([適合度検定データ],f_exp=[理論値])

In [71]:
ss.stats.chisquare([32,16,18,19,17,25,11,16,30,16],f_exp=[20,20,20,20,20,20,20,20,20,20])

Power_divergenceResult(statistic=20.6, pvalue=0.014549902304087554)

## Wilcoxonの順位和検定  
- 2グループの標本について、それぞれの母集団の中央値に差があるかどうかの検定
- 今までの様に分布を特定のものに限定しない  
- ２つの分布の形は同じそうだが、位置がずれているか？の判断に特に有効  
- 標本の値そのものではなく、順位に置き換えて検定統計量を求めるのが特徴  
ss.mannwhitneyu([data1],[data2], alternative="two-sided")  
<-alternativeパラメータ("two-sided","less","greater")
> 東京と大阪の住人を無作為に8人ずつ選んで、内閣支持に関するアンケートを行った。
5: 支持する、4:やや支持する、3:どちらとも言えない、2:やや支持しない、1:支持しない
以下の結果から、東京と大阪で内閣に対する意見は異なっているか検定せよ。  

|東京|大阪|
|:--:|:--:|
|4|3|
|3|3|
|3|1|
|2|5|
|1|2|
|3|5|
|4|1|

In [16]:
ss.mannwhitneyu([7.5,6.3,6.0,8.0,9.2,6.0,7.5,4.5,5.5,6.0],[8.3,5.8,6.1,8.1,8.9,10.4,13.0,6.1,10.1
,9.4], alternative="two-sided")

MannwhitneyuResult(statistic=21.0, pvalue=0.03082895140322003)

## Wilcoxonの符号順位検定
- 対応が有る2グループの標本についての標本について、それぉれの母集団の中央値に差があるかどうかの検定
- 今までのように分布を特定のものに限定しない
- 2ツノ分布の形は同じそうだが、位置がずれているかの判断に特に有効
> n人の対象者に省エネ活動の指導を行いその前後での意識調査の結果を比較する
> ある新薬の効果を試すために被験者8名を集めた。まず8名のγ-GTP値を計測し、その後2ヶ月新薬を服用してもらい、2ヶ月後γ-GTP値を再計測した。結果は以下のとおりである。Wilcoxonの符号順位検定により、新薬に効果があるかどうかを有意水準5%で検定せよ。

|被験者番号|投薬前|投薬後|
|:--:|:--:|:--:|
|1|100|95|
|2|110|100|
|3|120|105|
|4|130|110|
|5|135|120|
|6|140|145|
|7|220|170|
|8|250|195|

In [18]:
ss.wilcoxon([7.5,6.3,6.0,8.0,9.2,6.0,7.5,4.5,5.5,6.0],[8.3,5.8,6.1,8.1,8.9,10.4,13.0,6.1,10.1
,9.4])

TypeError: wilcoxon() got an unexpected keyword argument 'alternative'

## Fisherの正確確率検定
- 2つのカテゴリーに分類されたデータの分析(標本数が小さい場合)
- 2 x 2分割表(2つの集団が2カテゴリーに分類されたデータを扱う場合、自由度は1)の2変数の間に統計学的に有意な関連があるかどうかを調べる
- 同じ状況で<b>標本の大きさが大きい場合</b>には統計量の標本分布が近似的にカイ二乗分布に等しくなるので<b>カイ二乗検定</b>が用いられる
- 標本の大きさが小さい（分割表のセルの期待値に10未満のものがある）場合や、表中の数値の偏りが大きい場合にはこの近似は不正確。この場合には正確確率検定が正確
- 逆にサンプルサイズが大きい場合や、数値の偏りが小さい場合にはカイニ検定が良い。
> 男女各5人にある政策の賛否をアンケートしたところ，以下の結果を得た．男女間で賛否に有意な差があると言えるか。

![男女表](images/図1.png)  

> 以下の様な表で、「甘いものがすきか嫌いか」と「虫歯の有無」の間に関連があるかどうかを検定せよ。
有意水準5%の両側検定を実施せよ。
![虫歯と甘いもの](images/図2.png)

In [19]:
dat_edit = np.array([[6,2],[3,8]])
odds, p = ss.fisher_exact(dat_edit)

print("odds ratio:" + str(odds))
print("p値:"+ str(p))

odds ratio:8.0
p値:0.0697785186949274


# 分散分析 二元配置分散分析

## ルビーン検定
集団が正規分布から乖離している場合に使用

In [ ]:
ss.levene()

## バートレット検定

In [ ]:
ss.bartlett()

## 対比較

### Turkey法（サンプルサイズ同じ）

In [ ]:
# データは多次元配置のnparray
A = np.array([])
B = np.array([])
C = np.array([])
D = np.array([])

data_arr = np.hstack((A,B,C,D))
ind_arr = np.reoeat(list("ABCD"), len(A))
print(pairwise_tukeyhsd(data_arr, ind_arr))

### Turkey-Kramer法（サンプルサイズ違ってもいい）
正規性が前提

In [76]:
'''A = [14, 15, 14, 16, 15, 17, 17]
B = [17, 16, 17, 16, 15, 18, 19, 15]
C = [18, 19, 20, 19, 17, 17]
D = [20, 21, 19, 20, 19, 22, 20]
E = [19, 20, 19, 17, 17, 17, 18]
'''

A = np.array([9.5,9.7,10.1,9.8,9.3])
B = np.array([10.1,10.5,9.6,9.3])
C = np.array([11.3,10.7,10.2])


def tukey_hsd( ind, *args ):
    data_arr = np.hstack( args )
    ind_arr = np.array([])
    for x in range(len(args)):
      ind_arr = np.append(ind_arr, np.repeat(ind[x], len(args[x])))
    print(pairwise_tukeyhsd(data_arr,ind_arr))

#tukey_hsd(list('ABC') , A,B,C) # 第1引数:名称のリスト, 第2引数以降: データ
if __name__ == "__main__":
    tukey_hsd(list("ABC"), A, B, C)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper reject
-------------------------------------------
  A      B     0.195    -0.647 1.037 False 
  A      C     1.0533   0.1366  1.97  True 
  B      C     0.8583  -0.1004 1.817 False 
-------------------------------------------


### Steel-Dwass test検定
正規性なしでも可  
ただし等分散性は必要

In [1]:
#!/usr/bin/env python

"""
Here, we use pstrurng() in statsmodels that can be imported as below:

        from statsmodels.stats.libqsturng import psturng

pstrung(q, r, v) is used to evaluates the probability from 0 to q for a
            studentized range having v degrees of freedom and r samples.

  Definition of the function:

    def psturng(q, r, v):
        Parameters
        ----------
        q : (scalar, array_like)
            quantile value of Studentized Range
            q >= 0.
        r : (scalar, array_like)
            The number of samples
            r >= 2 and r <= 200
            (values over 200 are permitted but not recommended)
        v : (scalar, array_like)
            The sample degrees of freedom
            if p >= .9:
                v >=1 and v >= inf
            else:
                v >=2 and v >= inf

        Returns
        -------
        p : (scalar, array_like)
            1. - area from zero to q under the Studentized Range
            distribution. When v == 1, p is bound between .001
            and .1, when v > 1, p is bound between .001 and .9.
            Values between .5 and .9 are 1st order appoximations.

Because it uses interpolation from R data, the probability of CDF is not
exactly same as the one from R's ptukey().
E.g.,
   Python:
    >>print(psturng(3.997799075635331, 10, 465.4956))
    0.13000739280348494

   R:
    > 1 - ptukey(3.997799075635331,  10, 465.4956)
    0.1305053
"""

import pandas as pd
import numpy as np
import scipy.stats as ss

def flatten(*iterables):
    """ Recursive flatten a nested list """
    for s in iterables:
        try:
            it_is = iter(s)
        except TypeError:
            yield s
        else:
            for i in it_is:
                for j in flatten(i):
                    yield j

def steel_dwass(data, rank_method='average'):
    """
    Steel-Dwass pairwise ranking test

    This function rewritten in Python refers to

    The Steel Dwass method performs the multiple comparisons whilst
    controlling the overall experiment-wise error rate
    (it is the non-parametric equivalent to the Tukey All-Pairs method)
     adapted from http://aoki2.si.gunma-u.ac.jp/R/Steel-Dwass.html


    Args:
        data(pandas dataframe):
            data with more than 3 groups
    Kwargs:
        rank_method (str, optional):
            The method used to assign ranks to tied elements.
            The options are ‘average’, ‘min’, ‘max’, ‘dense’ and ‘ordinal’.

        alpha (float): the significant level. Default is 0.05
    Returns:
        dataframe
    """
    import itertools as it
    from statsmodels.stats.libqsturng import psturng

    r_method_types = ['average', 'min', 'max', 'dense', 'ordinal']
    if rank_method not in r_method_types:
        raise ValueError("Unknown rank method: "
                         "it should be 'average', "
                         "'min', 'max', 'dense', 'ordinal'")

    # The rows are used as samples.
    cols = data.index
    ngroups = len(cols)
    if ngroups < 3:
        raise ValueError("The input data should more than 3 groups")

    # Make combinations
    combs = list(it.combinations(cols, 2))
    static = []

    for d in combs:
        # flatten to a list
        d1 = list(flatten(data.loc[d[0]].values))
        d2 = list(flatten(data.loc[d[1]].values))
        # length for each rpws
        d1_num = len(d1)
        d2_num = len(d2)
        # Total number of elements
        N = d1_num + d2_num
        # concatenate two lists
        d1_d2 = d1 + d2
        # get the rank for the combined list
        r = ss.rankdata(d1_d2, method=rank_method)
        
        R = sum(r[:d1_num])
        E = d1_num * (N + 1) / 2
        V = d1_num * d2_num/(N * (N - 1)) * (sum(r**2) - N * (N + 1)**2/4)
        t = abs(R - E) / np.sqrt(V)
        # Tukey cdf
        p_val = psturng(t * np.sqrt(2), ngroups, np.inf)
        static.append([t, p_val])

    return pd.DataFrame(static, columns=['t', 'p'], index=combs)



if __name__ == '__main__':
    # The sample data as shown in R
    
    data = [[5,4,6,3,3,7,6,5,3,5],
            [8,4,3,3,7,9,8,7,3,4],
            [7,6,8,9,10,9,8,9,7,8]]
        
#    data = [[6.9, 7.5, 8.5, 8.4, 8.1, 8.7, 8.9, 8.2, 7.8, 7.3, 6.8],
#            [9.6, 9.4, 9.5, 8.5, 9.4, 9.9, 8.7, 8.1, 7.8, 8.8],
#            [5.7, 6.4, 6.8, 7.8, 7.6, 7.0, 7.7, 7.5, 6.8, 5.9],
#            [7.6, 8.7, 8.5, 8.5, 9.0, 9.2, 9.3, 8.0, 7.2, 7.9, 7.8]]
    # add row indices
    inx = ['g1', 'g2', 'g3']
    data = np.asarray(data)
    # Convert to dataframe
    #
    df = pd.DataFrame(data, index=inx)
    # call the self-defined function
    mc = steel_dwass(df)
    print(df)
    print(mc)


    0  1  2  3   4  5  6  7  8  9
g1  5  4  6  3   3  7  6  5  3  5
g2  8  4  3  3   7  9  8  7  3  4
g3  7  6  8  9  10  9  8  9  7  8
                 t         p
(g1, g2)  0.808763  0.680859
(g1, g3)  3.585362  0.001000
(g2, g3)  2.268701  0.060304


### Kruskal-Wallis one-way analysis of variance
![Kruskal](images/kkk.png)

In [ ]:
res = ss.kruskal([],[],[])
print(res)

In [ ]:
mu_0 = 1008
